<a href="https://colab.research.google.com/github/konstantin224/Classify_and_NER_vacancy_models/blob/main/%D0%98%D0%B7%D0%B2%D0%BB%D0%B5%D1%87%D0%B5%D0%BD%D0%B8%D0%B5_%D0%B8%D0%BC%D0%B5%D0%BD%D0%BE%D0%B2%D0%B0%D0%BD%D0%BD%D1%8B%D1%85_%D1%81%D1%83%D1%89%D0%BD%D0%BE%D1%81%D1%82%D0%B5%D0%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install spacy[transformers]

In [ ]:
import torch
import json
import pandas as pd
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
import spacy_transformers
import spacy

from spacy.util import filter_spans
from spacy.tokens import DocBin
from tqdm import tqdm

**Загружаем данные**

In [ ]:
with open('NER_data.json') as f:
    data = json.load(f)

**Выгружаем данные из JSON формата в словарь**

In [ ]:
training_data = {'classes' : ['REQUIREMENTS', "RESPONSIBILITIES", "OFFER"], 'annotations' : []}
for example in data:
  temp_dict = {}

  text = example['message_text']


  temp_dict['text'] = example['message_text']


  temp_dict['entities'] = []
  for annotation in example['label']:
    start = annotation['start']
    end = annotation['end']
    label = annotation['labels'][0].upper()
    temp_dict['entities'].append((start, end, label))
  training_data['annotations'].append(temp_dict)

In [ ]:
def trim_entity_spans(data: list) -> list:
    """

    Удаляем пустые пробелы, табуляцию в начале и конце разметки.

    """

    cleaned_data = []
    for sent in data["annotations"]:
        temp_dict = {}
        temp_dict['text'] = sent['text']
        temp_dict['entities'] = []

        text = sent["text"]
        entities = sent['entities']

        for start, end, label in entities:
            valid_start = start
            valid_end = end
            while valid_start < len(text) and (text[valid_start].startswith(' ') or
                                               text[valid_start].startswith('\n') or
                                               text[valid_start].startswith('\xa0') or
                                               text[valid_start].startswith('\t')):
                valid_start += 1
            while valid_end > 1 and (text[valid_end - 1].endswith(" ") or
                                     text[valid_end - 1].endswith("\n") or
                                     text[valid_end - 1].endswith("\xa0") or
                                     text[valid_end - 1].endswith("\t")):
                valid_end -= 1
            temp_dict['entities'].append((valid_start, valid_end, label))

        cleaned_data.append(temp_dict)

    return cleaned_data

In [ ]:
training_data = trim_entity_spans(training_data)

In [ ]:
nlp = spacy.blank("ru")
doc_bin = DocBin()

**Инициализируем предобученную модель, которая решает задачу NER с помощью трансформера**

In [ ]:
! python -m spacy init fill-config base_config.cfg config.cfg

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
for training_example  in tqdm(training_data):
    text = training_example['text']
    labels = training_example['entities']
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents
    doc_bin.add(doc)

doc_bin.to_disk("training_data.spacy")

100%|██████████| 433/433 [00:01<00:00, 357.83it/s]


**Запускаем обучение модели**

In [ ]:
! python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy --gpu-id 0

ℹ Saving to output directory: .
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
tokenizer_config.json: 100% 48.0/48.0 [00:00<00:00, 265kB/s]
config.json: 100% 625/625 [00:00<00:00, 3.24MB/s]
vocab.txt: 100% 872k/872k [00:00<00:00, 3.57MB/s]
tokenizer.json: 100% 1.72M/1.72M [00:00<00:00, 5.90MB/s]
model.safetensors: 100% 672M/672M [00:02<00:00, 252MB/s]
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipel

**Выводим справку по ошибкам и предупреждениям**

In [ ]:
! python -m spacy debug data ./config.cfg --paths.train ./training_data.spacy --paths.dev ./training_data.spacy

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(

============================ Data file validation ============================
✔ Pipeline can be initialized with data
✔ Corpus is loadable

=============================== Training stats ===============================
Language: ru
Training pipeline: transformer, ner
374 training docs
374 evaluation docs
⚠ 373 training examples also in evaluation data
⚠ Low number of examples to train a new pipeline (374)

============================== Vocab & Vectors ==============================
ℹ 87349 total word(s

**Подгружаем лучшую из моделей и даём ей на вход текст**

In [ ]:
nlp_ner = spacy.load("model-best")

In [ ]:
txt = "​Администратор-экскурсовод | Музей Эпос Калевала, Сортавала | от 25000 рублей\
Ищут человека, неравнодушного к северной красоте Карелии, который готов уехать на край света и с упоением рассказывать о тайнах местных минералов\
Какие требования:\
- Неподдельный интерес к природе сурового Севера;\
- Базовые познания в минералогии и желание приумножать свои знания;\
- Умение рассказывать так, чтобы все ждали продолжения, затаив дыхание;\
- Хорошо поставленная грамотная речь;\
- Опыт работы в туристической сфере будет преимуществом.\
Какие обязанности:\
- Знакомство гостей музея с минералогическими богатствами Карельского края и местной арт-галереей;\
- Проведение индивидуальных и групповых экскурсий;\
- Организация мастер-классов и обучающих мероприятий;\
- Продажа уникальных сувениров из лавки при музее.\
\
Какие условия:\
- Обучение и помощь в освоении со стороны музея;\
- Испытательный срок 2 месяца, повышение заработной платы после успешного прохождения;\
- Гибкий график с плавающими выходными;\
- Работа в одном из самых живописных краев России.\
\
Откликнуться на вакансию можно, отправив резюме на почту best.sortavala@gmail.com"

In [ ]:
doc = nlp_ner(txt)

colors = {"RESPONSIBILITIES": "#F67DE3", "REQUIREMENTS": "#7DF6D9", "OFFER":"#f20505"}
options = {"colors": colors}

**Результат работы модели**

In [ ]:
spacy.displacy.render(doc, style="ent", options= options, jupyter=True)

**Метрики модели**

В данном случае NER $P$, NER $R$, NER $F$ - это точность, полнота и $F_1-score$ соответственно.

In [ ]:
! spacy evaluate model-best/ training_data.spacy --gpu-id 0

ℹ Using GPU: 0
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(

================================== Results ==================================

TOK     100.00
NER P   94.10 
NER R   91.57 
NER F   92.82 
SPEED   5145  


=============================== NER (per type) ===============================

                       P       R       F
RESPONSIBILITIES   94.05   93.05   93.55
REQUIREMENTS       93.73   90.62   92.15
OFFER              94.68   91.35   92.99

